In [5]:
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp
import kfp.components as comp

In [2]:
@dsl.pipeline(name='load-predict-pipeline-logreg')
def pipeline(project_id='loan-predict'):
    preprocessor = dsl.ContainerOp(
        name='preprocessor',
        image='praveen049/loan-predict-logreg-preproc',
        command=['python', 'preprocessor.py'],
        arguments=[
            '--output-x', '/x.pkl',
            '--output-y', '/y.pkl',
    ],
        file_outputs={
            'x-output': '/x.pkl',
            'y-output': '/y.pkl',
        }
    )
    trainer = dsl.ContainerOp(
        name='trainer',
        image='praveen049/loan-predict-logreg-train',
        command=['python', 'train.py'],
        arguments=[
            '--input_x_path_file', dsl.InputArgumentPath(preprocessor.outputs['x-output']),
            '--input_y_path_file', dsl.InputArgumentPath(preprocessor.outputs['y-output']),
            '--output_model', '/model.pkl',
            '--output_model_path_file', '/model.txt',
        ],
        file_outputs={
            'model': '/model.pkl',
        }
    )
    trainer.after(preprocessor)

In [3]:
compiler.Compiler().compile(pipeline, 'load-predict-pipeline-logreg.zip')

In [6]:
client = kfp.Client()

In [7]:
experiment = client.get_experiment(experiment_name="NMGCPExp")

In [8]:
run_result = client.run_pipeline(experiment.id, 'test-2', 'load-predict-pipeline-logreg.zip')